# Masterclass Capstone Exercise Notebook

# Table of Contents
* [Introduction](#Introduction)
* [Experiment Setup](#Experiment-Setup)
* [Exercises](#Exercises)

# Introduction

This experiment notebook accompanies the cadCAD Edu "cadCAD Masterclass: Ethereum Validator Economics" practical exam for Section 4. Questions are numbered and named as on the course exam webpage for Section 4.

Where code needs to be completed or filled in to answer a specific question, this will be indicated as follows:
```python
#### Fill in the missing code ####
# Assign the value 5 to variable x
x = 
##################################
```

By running the code cell for the specific question, the answer will be printed out as `answer=...` at the end of the cell if completed correctly, and you can then select the corresponding multiple choice value on the exam webpage for the relevant question. You can also go to the end of the exercises section of this notebook to see all the completed answers.

# Experiment Setup

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime

import experiments.notebooks.visualizations as visualizations
import model.constants as constants
from experiments.run import run
from experiments.utils import display_code
from model.types import (
    ETH,
    USD_per_ETH,
    Stage,
)

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Create a simulation for each analysis
simulation = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
# Experiment configuration
DELTA_TIME = constants.epochs_per_day  # epochs per timestep
SIMULATION_TIME_MONTHS = 12 * 5  # number of months
TIMESTEPS = constants.epochs_per_month * SIMULATION_TIME_MONTHS // DELTA_TIME

eth_staked: ETH = 7_487_136
number_of_active_validators: int = eth_staked // 32
eth_supply: ETH = 117_386_100
eth_block_rewards_mean: ETH = 13527

initial_state_overrides = {
    'eth_supply': eth_supply,
    'eth_staked': eth_staked,
    'number_of_active_validators': number_of_active_validators,
    'number_of_awake_validators': number_of_active_validators,
}

parameter_overrides = {
    'stage': [Stage.ALL],
    'date_start': [datetime(2021,9,4)],
    'validator_process': [lambda _run, _timestep: 3],
    'daily_pow_issuance': [eth_block_rewards_mean],    
}

simulation.model.initial_state.update(initial_state_overrides)
simulation.model.params.update(parameter_overrides)
simulation.timesteps = TIMESTEPS

# Exercises

## Exercise 0:
An illustrative/example question: Set the value for the System Parameter `dt` to the `DELTA_TIME` variable, execute the experiment, and find the maximum value for the `eth_supply` State Variable.

In [ ]:
#### Fill in the missing code ####

# Set the System Parameter value as defined in the question
simulation.model.params.update({
    'dt': [DELTA_TIME]
})

##################################

# Experiment execution
df_0, _exceptions = run(simulation)

In [ ]:
eth_supply_max = df_0['eth_supply'].max()

In [ ]:
# Answer
answer_0 = round(eth_supply_max)
print(f'{answer_0=}')

## Exercise 1:

Configure a parameter sweep of the two values `None` and `524_288` for the `MAX_VALIDATOR_COUNT` System Parameter, execute the experiment, and determine the difference in `total_revenue_yields` at the last timestep.

In [ ]:
#### Fill in the missing code ####

# Create a deepcopy of the `simulation` object
# and assign it to the simulation for this question, `simulation_1`
simulation_1 = copy.deepcopy()

# Create a parameter sweep as defined in the question
simulation_1.model.params.update({
    'MAX_VALIDATOR_COUNT':
})

##################################

# Experiment execution
df_1, _exceptions = run(simulation_1)

In [ ]:
absolute_difference = abs(df_1.query('subset == 0')['total_revenue_yields_pct'].iloc[-1] - df_1.query('subset == 1')['total_revenue_yields_pct'].iloc[-1])

In [ ]:
# Answer
answer_1 = round(absolute_difference, 2)
print(f'{answer_1=}')

In [ ]:
px.line(df_1, x='timestamp', y='total_revenue_yields_pct', facet_col='subset')

## Exercise 2:

What is the total profit of a single validator in the DIY Hardware Validator Environment over the first year?

In [ ]:
simulation_2 = copy.deepcopy(simulation)

# Experiment execution
df_2, _exceptions = run(simulation_2)

In [ ]:
df_2 = df_2.set_index('timestamp', drop=False)

individual_diy_hardware_profit = df_2.first("365D")[
    #### Fill in the missing code ####
    ''
    ##################################
] / df_2['diy_hardware_validator_count']

#### Fill in the missing code ####
sum_of_individual_diy_hardware_profit = individual_diy_hardware_profit.
####################################

In [ ]:
# Answer
answer_2 = round(sum_of_individual_diy_hardware_profit)
print(f'{answer_2=}')

In [ ]:
df_2['cumulative_individual_diy_hardware_profit'] = (df_2['diy_hardware_profit'] / df_2['diy_hardware_validator_count']).transform('cumsum')
px.line(df_2.first('365D'), x='timestamp', y='cumulative_individual_diy_hardware_profit')

## Exercise 3:

If the base fee (System Parameter `base_fee_process`) varies from `0` to `200` Gwei per gas, at which base fee value does the ETH supply become deflationary (i.e. negative inflation rate)?

In [ ]:
simulation_3 = copy.deepcopy(simulation)

#### Fill in the missing code ####
base_fee_start = 0
base_fee_stop = 200
number_of_samples = 20
##################################

base_fee_process_samples = np.linspace(start=base_fee_start, stop=base_fee_stop, num=number_of_samples)

simulation_3.model.params.update({
    'dt': [constants.epochs_per_year],
    'base_fee_process': [
        #### Fill in the missing code ####
        lambda run, _timestep: [run - 1]
        ##################################
    ]
})

simulation_3.timesteps = 1
#### Fill in the missing code ####
simulation_3.runs =
##################################

# Experiment execution
df_3, _exceptions = run(simulation_3)

In [ ]:
negative_inflation_base_fee_index = df_3.query('supply_inflation < 0')['run'].iloc[0]
negative_inflation_base_fee = base_fee_process_samples[negative_inflation_base_fee_index]

In [ ]:
# Answer
answer_3 = round(negative_inflation_base_fee)
print(f'{answer_3=}')

## Exercise 4:

If only 2/3 of all active validators are online, what change in total validator revenue can we expect over 1 year?

In [ ]:
simulation_4 = copy.deepcopy(simulation)

simulation_4.model.params['validator_uptime_process'].append(
    #### Fill in the missing code ####
    
    ##################################
)
simulation_4.timesteps = constants.epochs_per_year // DELTA_TIME

# Experiment execution
df_4, _exceptions = run(simulation_4)

In [ ]:
change_in_total_revenue = abs(df_4.query('subset == 0')['total_revenue'].sum() - df_4.query('subset == 1')['total_revenue'].sum())

In [ ]:
answer_4 = round(change_in_total_revenue)
print(f'{answer_4=}')

## Exercise 5:

Assuming no new validators stake / are activated, and with a base fee of 200 Gwei per gas, in which year will we have hypothetically burnt 50% of the total ETH supply?

In [ ]:
simulation_5 = copy.deepcopy(simulation)

simulation_5.model.params.update({
    'dt': [constants.epochs_per_year],
    #### Fill in the missing code ####
    'validator_process': ,
    'base_fee_process': ,
    ##################################
})

#### Fill in the missing code ####
eth_supply_initial_state = simulation_5.model.initial_state['']
##################################

# Experiment execution
model_generator = iter(simulation_5.model)

model = None
state = {}

while state.get('eth_supply', eth_supply_initial_state) >= eth_supply_initial_state * 0.5:
    # Step to next state
    model = next(model_generator)
    # Update state
    state = model.state

year = state['timestamp'].year

In [ ]:
# Answer
answer_5 = year
print(f'{answer_5=}')

## Exercise Answers

All completed [exercises](#Exercises) answers will be displayed here once their respective cells have been executed:

In [ ]:
for k,v in list(locals().items()):
    if 'answer' in k: print(f'{k}={v}')